In [1]:
import pygrib
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

from icedef import metocean

In [2]:
grbs = pygrib.open('/home/evankielley/Downloads/ofs_atl.t00z.N000.20150420.3d.grb.grib2')

In [3]:
for grb in grbs[:2]:
    print(grb)

RuntimeError: b'Key/value not found'

In [ ]:
grbs

In [ ]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)
df = pd.read_csv('/home/evankielley/Downloads/eng-hourly-04012015-04302015.csv', 
                  usecols=['Date/Time', 'Wind Spd (km/h)', 'Wind Dir (10s deg)'], skiprows=15)

In [ ]:
df['Date/Time'] = pd.to_datetime(df['Date/Time'])
df['Wind Dir (deg)'] = df['Wind Dir (10s deg)'] * 10
df['Wind Spd (m/s)'] = df['Wind Spd (km/h)'] * (1000 / 3600)
df['Wind Dir (rad)'] = np.deg2rad(df['Wind Dir (deg)'])
df['eastward_wind_velocity'] =  -df['Wind Spd (m/s)'] * np.sin(df['Wind Dir (rad)'])
df['northward_wind_velocity'] =  -df['Wind Spd (m/s)'] * np.cos(df['Wind Dir (rad)'])

In [ ]:
df

In [ ]:
start_date = np.datetime64('2015-04-19')
end_date = np.datetime64('2015-04-20')
date_bounds = start_date, end_date
atm = metocean.NARRAtmosphere(date_bounds)

In [ ]:
plt.imshow(atm.eastward_wind_velocities[0, :, :])

In [ ]:
start_index = np.where(df['Date/Time'] > atm.dataset.time.values[0])[0][0]
end_index = np.where(df['Date/Time'] < atm.dataset.time.values[-1])[0][-1]

In [ ]:
latitude, longitude = 47.6212, -52.7424  # St. John's International Airport

d = {'time': [], 'obs_u': [], 'obs_v': [], 'mod_u': [], 'mod_v': []}

for i in range(start_index, end_index + 1):

    time = np.datetime64(df['Date/Time'][i])
    d['time'].append(time)
    d['obs_u'].append(df['eastward_wind_velocity'][i])
    d['obs_v'].append(df['northward_wind_velocity'][i])
    d['mod_u'].append(float(atm.eastward_wind_velocities.interp(time=time, latitude=latitude, longitude=longitude, 
                                                         assume_sorted=True).values))
    d['mod_v'].append(float(atm.northward_wind_velocities.interp(time=time, latitude=latitude, longitude=longitude, 
                                                         assume_sorted=True).values))

In [ ]:
d['time'][-1]

In [ ]:
gap = 2
plt.figure(figsize=(14, 4))
plt.quiver(d['obs_u'][::gap], d['obs_v'][::gap], width=5e-3, scale=100, color='black', label='observed')
plt.quiver(d['mod_u'][::gap], d['mod_v'][::gap], width=5e-3, scale=100, color='red', label='predicted')
plt.title('St. John\'s International Airport Wind Velocities from {} to {}'.format(
    pd.to_datetime(str(d['time'][0])).strftime('%Y.%m.%d.%H.%M'), 
    pd.to_datetime(str(d['time'][-1])).strftime('%Y.%m.%d.%H.%M')))
plt.legend()
plt.show()